In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import re
import warnings
warnings.filterwarnings('ignore')

# importing the data form csvs

In [28]:
ag_all_bh = pd.read_csv('ag_all_bh.csv')
ag_col_bh = pd.read_csv('ag_col_bh.csv')
ag_merge_bh = pd.read_csv('ag_merge_bh.csv')
ag_col = pd.read_csv('ag_col.csv')
ag_merge = pd.read_csv('ag_merge.csv')


In [29]:
ag_col_bh

,time_x,collision_type,idm,mm,id1,m1,id2,m2,r_x,typem,...,bh_mass,bh_spin,birth_kick,model,total_mass,mass_difference,collision_cluster,logm1,logbig,collision_type_hdbscan
0,11.565179,single-single,1680002,33.8986,1680001,33.7704,470820,0.129842,0.038822,4,...,16.09650,0.0,1.00000,data\N16rv0.5rg2.0Z0.002,33.900242,33.640558,3,3.519585,3.519585,632
1,7.978654,single-single,1680061,133.8740,1680060,179.9130,299190,0.394966,0.013368,4,...,29.70350,0.0,0.00000,data\N16rv0.5rg2.0Z0.002,180.307966,179.518034,3,5.192473,5.192473,609
2,3.710148,single-single,1680133,110.5980,1680132,110.4200,261872,0.179723,0.025530,4,...,31.67270,0.0,0.00000,data\N16rv0.5rg2.0Z0.002,110.599723,110.240277,3,4.704291,4.704291,528
3,4.100341,single-single,1680220,41.3707,1680219,41.2697,328483,0.105145,0.052498,4,...,30.70980,0.0,1.00000,data\N16rv0.5rg2.0Z0.002,41.374845,41.164555,3,3.720129,3.720129,463
4,3.849036,single-single,1680292,52.0011,1680291,51.8734,137163,0.133144,0.041029,4,...,36.55280,0.0,1.00000,data\N16rv0.5rg2.0Z0.002,52.006544,51.740256,3,3.948806,3.948806,386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25558,7.467068,single-single,867397,27.4788,27397,27.2059,34148,0.274166,0.809076,4,...,12.91490,0.0,196.38100,data\N8.0rv4.0rg8.0Z0.002,27.480066,26.931734,3,3.303434,3.303434,278
25559,8.195550,single-single,935010,21.1569,95010,21.0637,260784,0.094146,1.787344,4,...,11.89070,0.0,89.37041,data\N8.0rv4.0rg8.0Z0.002,21.157847,20.969553,3,3.047551,3.047551,488
25560,8.195550,single-single,1021125,23.4548,239877,23.3603,181125,0.095204,2.219268,4,...,11.48710,0.0,149.45210,data\N8.0rv4.0rg8.0Z0.002,23.455504,23.265096,3,3.151038,3.151038,635
25561,6.479118,single-single,901838,26.2744,61838,26.0639,69692,0.210457,0.995484,1,...,7.63069,0.0,1.00000,data\N8.0rv4.0rg8.0Z0.02,26.274357,25.853443,1,3.260551,3.260551,75


## Grouping 

## multiindex data frame test

In [ ]:
ag_col_bh['int_type'] = pd.Categorical(
    np.select([
        (ag_col_bh['type1'].isin([0,1]) & ag_col_bh['type2'].isin([0,1])),
        (ag_col_bh['type1'].isin(range(2,10)) & ag_col_bh['type2'].isin(range(2,10))),
        ((ag_col_bh['type1'].isin([0,1]) & ag_col_bh['type2'].isin(range(2,10))) | 
         (ag_col_bh['type1'].isin(range(2,10)) & ag_col_bh['type2'].isin([0,1]))),
        ((ag_col_bh['type1'] >= 10) | (ag_col_bh['type2'] >= 10))
    ], ['MS+MS', 'Giant+Giant', 'MS+Giant', 'Other'], default='Unknown'),
    categories=['MS+MS', 'MS+Giant', 'Giant+Giant', 'Other', 'Unknown'],
    ordered=True
)


In [45]:
df_processed = ag_col_bh.copy()

# Mass ratio categories
df_processed['q'] = pd.cut(
    df_processed['mass_ratio'], 
    bins=[0, 0.01, 0.1, 1.0], 
    labels=['0.01', '0.1', '1'],
    include_lowest=True
)

# Type categories
df_processed['type'] = df_processed['typem'].apply(
    lambda x: 'MS' if x in [0, 1] else 'Giant' if x in [2, 3, 4, 5, 6, 7, 8, 9] else 'Other'
)

# Larger mass categories
df_processed['pm'] = pd.cut(
    df_processed['larger_mass'], 
    bins=[0,10, 50, 100, 150], 
    labels=['10','50', '100', '150'],
    include_lowest=True
)

# Create MultiIndex DataFrame
col = df_processed.set_index(['q', 'type', 'pm','int_type'])

In [52]:
q_list = ['0.01', '0.1', '1']
type_list=['MS','Giant']
pm_list= ['10','50', '100', '150']

groups = {}
names = []
for q in q_list:
    for type in type_list:
        variable_name = f"col_{q}_{type}"
        names.append(variable_name)
        groups[variable_name] = col.loc[q,type,:,:]
        
groups['col_1_MS']
names

['col_0.01_MS',
 'col_0.01_Giant',
 'col_0.1_MS',
 'col_0.1_Giant',
 'col_1_MS',
 'col_1_Giant']